In [1]:
!pip install unidecode
!pip install transformers==4.2.2
!pip install pyreadr
!pip install unidecode
!pip install ftfy
!pip install git+https://github.com/legalnlp21/legalnlp

     |████████████████████████████████| 241 kB 7.4 MB/s 
     |████████████████████████████████| 1.8 MB 7.5 MB/s 
     |████████████████████████████████| 2.9 MB 63.9 MB/s 
     |████████████████████████████████| 895 kB 69.3 MB/s 
     |████████████████████████████████| 409 kB 7.1 MB/s 
     |████████████████████████████████| 64 kB 2.6 MB/s 
  Created wheel for ftfy: filename=ftfy-6.0.3-py3-none-any.whl size=41933 sha256=b4b6f98934bb09479b2fa0c9eb5729c7c27cbc1745ce7799b09644779b9c552d
  Stored in directory: /root/.cache/pip/wheels/19/f5/38/273eb3b5e76dfd850619312f693716ac4518b498f5ffb6f56d
Successfully built ftfy
  Cloning https://github.com/legalnlp21/legalnlp to /tmp/pip-req-build-z0jxzmgd
  Running command git clone -q https://github.com/legalnlp21/legalnlp /tmp/pip-req-build-z0jxzmgd
  Created wheel for legalnlp: filename=legalnlp-1.0.0-py3-none-any.whl size=4242 sha256=4f9807603d1d3a63ba71f88466e83152aed5f09b2ce650dd0a037f7c534d4728
  Stored in directory: /tmp/pip-ephem-wheel-cache

In [2]:
from IPython.display import Image
from IPython.display import clear_output
import pickle
import random
import unidecode
import re
import ftfy
import os
import copy
import torch
import torch.nn as nn
import torch.utils.data as tdata
import torch.optim as optim
import transformers
from transformers import AutoModel, AutoTokenizer, AutoConfig
from transformers import BertForPreTraining, BertModel, BertTokenizer, BertForMaskedLM, BertForNextSentencePrediction, BertForQuestionAnswering
# manipulação numérica e de dataframes
import numpy as np
import pandas as pd
# gráficos e ajustes visuais
import matplotlib.pyplot as plt
import textwrap
from tqdm import tqdm

In [3]:
# Import functions of legalnlp
from legalnlp.get_premodel import *
from legalnlp.clean_functions import *

In [4]:
#Downloading BERTikal
get_premodel('bert')

In [5]:
%%time
#Loading dataset and pre-trained model
data=pd.read_csv('https://raw.githubusercontent.com/legalnlp21/legalnlp/main/demo/data_base.csv')
data.drop(columns=['Unnamed: 0'],inplace=True)
data['text'] = data['text'].apply(lambda x:clean_bert(x))
model = '/content/BERTikal/BERTikal'
tokenizer= '/content/BERTikal/BERTikal/vocab.txt'
data

CPU times: user 406 ms, sys: 12.3 ms, total: 418 ms
Wall time: 715 ms


In [6]:
def extract_features_bert(path_model, path_tokenizer, data,gpu=True):
    if gpu:
        device = torch.device('cuda')
        os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
        bert_tokenizer = BertTokenizer.from_pretrained(path_tokenizer, do_lower_case=False)
        bert_model = BertModel.from_pretrained(path_model).to(device)
        bert = data.apply(lambda x: bert_tokenizer.encode(x, add_special_tokens=True, max_length=512, truncation=True))
        data_text = list(data)
        encoded_inputs = bert_tokenizer(data_text, padding=True, truncation=True, max_length=512, return_tensors="pt")
        input_ids = encoded_inputs['input_ids'].to(device)
        clear_output()
        # Creating our feature vector
        features = []
    
        # Applying the pre-trained model to each sentence and adding it to our vector
    
        for i in tqdm(range(len(data)), mininterval=40, maxinterval=500):
    
            with torch.no_grad():
    
                last_hidden_states = bert_model(
                    input_ids[i:(i+1)])[1].cpu().numpy().reshape(-1).tolist()
            features.append(last_hidden_states)
    else:
        bert_tokenizer = BertTokenizer.from_pretrained(path_tokenizer, do_lower_case=False)
        bert_model = BertModel.from_pretrained(path_model)
        bert = data.apply(lambda x: bert_tokenizer.encode(
            x, add_special_tokens=True, max_length=512, truncation=True))
        encoded_inputs = bert_tokenizer(
            data_text, padding=True, truncation=True, max_length=512, return_tensors="pt")
        input_ids = encoded_inputs['input_ids']
        # Creating our feature vector
        features = []
    
       # Applying the pre-trained model to each sentence and adding it to our vector
    
        for i in tqdm(range(len(data))):
    
            with torch.no_grad():
    
                last_hidden_states = bert_model(
                    input_ids[i:(i+1)])[1].cpu().numpy().reshape(-1).tolist()
            features.append(last_hidden_states)
    features = np.array(features)
    features[:2]
    df_features = pd.DataFrame(features)
    return df_features



    # os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
    #device = torch.device('cuda:0')
features= extract_features_bert(model,tokenizer,data['text'])

100%|██████████| 5093/5093 [03:45<00:00, 22.54it/s]


In [7]:
features

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
0,-0.020649,0.062290,-0.036701,-0.129206,-0.030147,0.267448,0.985803,0.404766,-0.043943,-0.008192,-0.965343,-0.036896,0.174238,-0.086198,-0.253857,-0.178252,-0.129426,0.088896,0.983750,0.514608,0.003525,0.316241,-0.273740,0.133142,-0.276909,0.338380,0.248214,-0.211505,0.065028,0.215943,-0.362236,-0.894944,-0.902823,-0.003573,0.280066,-0.342072,-0.212170,0.014028,0.922013,0.341220,...,0.311519,-0.189640,0.348582,-0.146080,-0.178621,0.088065,-0.132068,-0.175349,0.073264,-0.192410,-0.300864,-0.139611,-0.987249,0.156078,0.037289,0.285448,0.115848,0.349242,-0.180449,0.178451,-0.258687,0.088996,0.491710,0.192816,-0.203070,-0.036393,0.169442,-0.012561,-0.784642,0.203821,-0.327722,0.221272,0.035829,0.120155,-0.377430,-0.018894,0.236955,0.198742,-0.326066,-0.407968
1,0.265664,-0.191833,0.123083,0.138307,0.115190,-0.151363,0.984313,-0.038230,0.335999,-0.397480,-0.664342,-0.121024,0.103919,0.105919,-0.371716,-0.017340,-0.228911,0.032581,0.594250,0.910806,0.266540,0.345002,-0.017102,0.088835,0.279707,0.069117,-0.144009,0.093980,-0.146020,-0.156663,-0.288577,-0.953372,-0.751004,-0.184736,0.067112,-0.132566,-0.080650,-0.068849,0.655235,0.105160,...,0.058616,0.122967,0.189384,0.181818,-0.044179,-0.145809,0.058292,0.321994,-0.081371,-0.164444,-0.043541,-0.187506,-0.922961,-0.002745,-0.071674,0.167595,0.030992,0.229105,0.096547,0.023741,0.874489,0.067064,-0.105497,0.140603,0.079882,-0.033177,0.272004,-0.134086,-0.773445,0.198007,-0.091300,0.110063,0.236758,0.600863,0.141063,-0.118244,0.057350,0.121923,-0.160961,-0.304954
2,-0.020429,-0.104498,-0.040864,0.071459,-0.093188,-0.020166,0.992611,0.244074,0.288444,-0.316070,-0.696078,0.072880,-0.022712,0.046194,-0.306239,-0.135350,0.020451,-0.212678,0.738498,0.820407,0.381373,-0.155491,-0.086790,-0.232232,0.078778,0.093693,-0.288019,0.022589,-0.082188,-0.407113,-0.074037,-0.933521,-0.918427,0.091494,0.070079,0.202597,0.323535,0.040485,0.841374,-0.119930,...,-0.292243,-0.039534,0.319895,0.089062,-0.080972,-0.196442,0.207313,0.040770,0.076264,-0.072229,0.051533,-0.046664,-0.962345,0.098780,0.188235,0.233839,0.001149,0.366119,0.068241,-0.133488,0.401989,0.181832,0.040236,-0.103024,0.021774,-0.294924,0.119631,-0.203095,-0.868799,0.023280,-0.041266,0.270103,0.348593,0.529465,-0.214764,-0.002462,0.268698,-0.037324,-0.027369,-0.252907
3,-0.147217,-0.146643,0.152111,0.066671,0.074747,0.512806,0.988143,0.266524,0.166316,0.039877,-0.952476,-0.057509,-0.091864,-0.083918,-0.123271,-0.213122,-0.076535,0.050128,0.939576,0.699231,0.547445,-0.116606,-0.129612,-0.077114,0.099442,-0.194523,0.122533,-0.127319,-0.099779,-0.071605,-0.030850,-0.939521,-0.927001,-0.098597,0.079869,-0.215021,-0.018590,-0.188348,0.939666,0.215551,...,0.147656,-0.006433,0.259502,0.070496,-0.035583,-0.042674,-0.165597,0.019068,-0.009598,-0.121617,0.026108,0.054056,-0.919466,-0.013944,-0.059075,0.173053,0.189487,0.091862,0.146744,0.281982,0.145236,0.201707,-0.100612,0.003693,-0.182819,-0.591287,-0.204799,-0.248201,-0.886959,-0.034516,-0.193701,0.136181,0.147152,-0.033698,0.207742,-0.194004,0.221950,-0.122347,0.002691,-0.487538
4,-0.203890,0.238025,0.106599,-0.207906,0.000219,0.024782,0.991711,0.305908,0.262042,-0.264231,-0.950009,0.098787,0.073179,0.018353,-0.106340,-0.172629,0.057338,-0.066193,0.958004,0.896513,0.400789,-0.049524,-0.458125,0.179687,0.143407,0.185427,0.162097,-0.067593,0.043588,-0.261461,0.025266,-0.869108,-0.892458,0.041983,-0.139930,-0.315681,0.311566,0.141892,0.864385,0.454063,...,-0.017940,-0.217046,0.312199,-0.007367,0.266694,0.328706,0.481016,-0.137123,-0.228023,0.029773,-0.022112,-0.227679,-0.909943,0.202506,-0.071558,0.306340,0.243157,0.227650,-0.235798,-0.089544,0.313984,0.362650,0.449550,-0.157221,-0.222304,0.090268,-0.190394,-0.269871,-0.687768,-0